## 00. 프로젝트 목적
- 본 프로젝트는 Pinecone vector store를 위한 연습 프로젝트입니다.

- [Pinecone 공식 홈페이지](https://docs.pinecone.io/integrations/langchain)
- [Pinecone 랭체인](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)

### 필요한 환경변수 로드

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os


# API 키 정보 로드
load_dotenv()

# 환경 변수에서 OpenAI API 키 읽어오기
openai_api_key = os.environ.get('OPENAI_API_KEY')

# 환경 변수에서 Pinecone API 키 읽어오기
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

## 01. PDF to Text
1. PDF 텍스트를 추출하는데 강점을 가진 함수
    - `extract_text_from_pdf`
    - 텍스트가 주를 이루고 있을 때, 사용한다.
2. PDF to Markdown 함수
    - `extract_markdown_from_pdf`
    - 표와 텍스트가 병합하여 사용되고 있을 때 사용하면 유용하다.

In [159]:
import sys
import os

# vector_store 폴더의 상위 디렉토리인 프로젝트 루트를 수동으로 지정합니다.
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from create_dataset.utils import extract_markdown

# PDF 파일에서 마크다운 추출
chunks = extract_markdown.extract_markdown_from_pdf('../create_dataset/source_data/pdf/prompt_test.pdf')

PDF 파일 로드 중: ../create_dataset/source_data/pdf/prompt_test.pdf
총 5개의 청크로 나누었습니다.
총 5개의 청크 중 5개의 유효한 청크를 추출했습니다.
총 5개의 청크 중 5개의 병합된 청크를 생성했습니다.
병합으로 0개의 청크가 줄었습니다.


In [153]:
chunks

['--- Chunk 1/5 ---\n### 과목 소개\n\n#\n\n#### ‘공통수학1’과 ‘공통수학2’는 수학에 대한 기초 소양과 학문적 이해를 기반으로 학생 스스로 자신의 적성을 개발하여 창의성을 갖춘 사람으로 성장하기 위해 수학의 여러 영역의 기본적인 내용을 학습하는 과목이다. 특히 ‘공통수학1’은 중학교 ‘변화와 관계’ 영역에서 학습한 다항식, 방정식, 부등식이 심화되고 다양한 유형으로 다루어지며, ‘자료와 가능성’ 영역에서 학습한 경우의 수가 순열과 조합을 활용하는 방법으로 체계화된다.\n\n#\n\n## 무엇을 배울까요?\n#\n\n### 40 [ 경상남도교육청]\n\n|범주 다항식 방정식과 부등식 지식·이해 경우의 수 행렬|Col2|내용 요소 • 다항식의 연산 • 나머지정리 • 인수분해|\n\n|---|---|---|\n||방정식과 부등식|• 복소수와 이차방정식 • 이차방정식과 이차함수 • 여러 가지 방정식과 부등식|\n||경우의 수|• 합의 법칙과 곱의 법칙 • 순열과 조합|\n||행렬|• 행렬과 그 연산|\n|과정·기능|• 다항식, 방정식과 부등식, 경우의 수, 행렬의 개념, 원리, 법칙이나 자신의 수학적 사고와 전략을 설명하기 • 수학적 절차를 수행하고 계산하기 • 적절한 전략을 사용하여 문제해결하기 • 이차방정식과 이차부등식을 이차함수와 연결하기 • 이차함수의 그래프와 직선의 위치 관계를 판단하기 • 다항식, 방정식과 부등식, 경우의 수, 행렬의 개념, 원리, 법칙, 성질을 탐구하기 • 방정식과 부등식 풀기 • 방정식과 부등식, 경우의 수, 행렬을 실생활과 연결하기 • 식과 그래프, 수학 기호, 행렬 등을 표현하기||\n\n-----\n\n#',
 '--- Chunk 2/5 ---\n## 과목 소개\n\n#\n\n#### ‘공통수학1’과 ‘공통수학2’는 수학에 대한 기초 소양과 학문적 이해를 기반으로 학생 스스로 자신의 적성을 개발하여 창의성을 갖춘 사람으로 성장하기 위해 수학의 여러 영역의 기본적인 내용을 학습하는 과목이다. 특히 ‘공

### 02. Text Embedding
- 변환된 텍스트를 벡터로 임베딩
- 텍스트 청크를 Langchain의 Document 객체로 변환하는 과정

In [5]:
from langchain.schema import Document

# Document 리스트로 변환
documents = [Document(page_content=chunk, metadata={"source": "your-source-name"}) for chunk in chunks]

# 각 청크에 'text' 메타데이터 추가
for chunk in documents:
    chunk.metadata['text'] = chunk.page_content

In [11]:
chunk

Document(metadata={'source': 'your-source-name', 'text': '-----'}, page_content='-----')

In [161]:
from tqdm import tqdm
from openai import OpenAI


# 클라이언트 초기화
client = OpenAI(api_key=openai_api_key)

# 텍스트 → 벡터로 변환
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-3-large",
        input=[text]
    )
    return response.data[0].embedding


embeddings = []
for chunk in tqdm(chunks, desc="🔍 임베딩 중"):
    embeddings.append(get_embedding(chunk))

🔍 임베딩 중:   0%|          | 0/5 [00:05<?, ?it/s]


KeyboardInterrupt: 

### 03. Pinecone Vector DB
- Index 생성

In [7]:
from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone as LangchainPinecone

# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=pinecone_api_key)

/opt/homebrew/Caskroom/miniconda/base/envs/bearable/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Pinecone index 확인
pc.list_indexes()

[
    {
        "name": "bearable-openai",
        "metric": "dotproduct",
        "host": "bearable-openai-x6xczpc.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": null
    }
]

In [ ]:
# 해당 코드는 한번만 합니다.

# Pinecone 인덱스 생성
index_name = "bearable-openai"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model": "text-embedding-ada-002",
            "field_map": {"text": "chunk_text"}
        }
    )

In [162]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain_pinecone import PineconeVectorStore

# OpenAI 임베딩 인스턴스 생성
embeddings = OpenAIEmbeddings(
    model='text-embedding-ada-002',
    openai_api_key=openai_api_key
)

# 문자열 리스트(chunks)가 있을 경우 Document 객체로 변환
documents = [Document(page_content=chunk) for chunk in chunks]

# PineconeVectorStore 생성
pinecone_database = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=embeddings,
    pinecone_api_key=pinecone_api_key,
    index_name="bearable-openai"
)

/opt/homebrew/Caskroom/miniconda/base/envs/bearable/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/jn/6v9ffl5x6gvf4y16kqq_wqgh0000gn/T/ipykernel_37545/2261209657.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


### 02. Text Embedding
- 변환된 텍스트를 벡터로 임베딩
- 텍스트 청크를 Langchain의 Document 객체로 변환하는 과정

In [163]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# ✅ 기본 Similarity Retriever
similarity_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # 가장 유사한 top-k 문서 반환
)

llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)

# QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=similarity_retriever,
    return_source_documents=True  # 어떤 chunk에서 답을 가져왔는지도 확인 가능
)

# 질문해보기
query = "공통수학 1 범주는 어떤게 있어?"
result = qa_chain(query)

print("Answer:", result["result"])
print("\nSource documents:")
for doc in result["source_documents"]:
    print(doc.metadata, doc.page_content[:100], "...")

NameError: name 'vectorstore' is not defined